In [ ]:
# -------------------------------
# 1. INSTALL & IMPORT DEPENDENCIES
# -------------------------------
!pip install -q kaggle
!pip install -q scikit-learn
!pip install -q -U keras-efficientnet-v2
!pip install -U keras-efficientnet-v2
!pip install -q efficientnet



import os
import numpy as np
import zipfile
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from efficientnet.tfkeras import EfficientNetB0

# -------------------------------
# 2. UPLOAD KAGGLE.JSON
# -------------------------------
from google.colab import files
files.upload()  # upload kaggle.json

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# -------------------------------
# 3. DOWNLOAD AND UNZIP DATASET
# -------------------------------
!kaggle datasets download -d navoneel/brain-mri-images-for-brain-tumor-detection
!unzip -q brain-mri-images-for-brain-tumor-detection.zip -d data

# -------------------------------
# 4. ORGANIZE DATASET (Train/Val/Test)
# -------------------------------
def organize_dataset(base_path='data/brain_tumor_dataset'):
    os.makedirs('dataset/train/yes', exist_ok=True)
    os.makedirs('dataset/train/no', exist_ok=True)
    os.makedirs('dataset/val/yes', exist_ok=True)
    os.makedirs('dataset/val/no', exist_ok=True)
    os.makedirs('dataset/test/yes', exist_ok=True)
    os.makedirs('dataset/test/no', exist_ok=True)

    yes_images = os.listdir(f'{base_path}/yes')
    no_images = os.listdir(f'{base_path}/no')

    yes_train, yes_test = train_test_split(yes_images, test_size=0.2, random_state=42)
    yes_train, yes_val = train_test_split(yes_train, test_size=0.2, random_state=42)

    no_train, no_test = train_test_split(no_images, test_size=0.2, random_state=42)
    no_train, no_val = train_test_split(no_train, test_size=0.2, random_state=42)

    for name, group in [('train', yes_train), ('val', yes_val), ('test', yes_test)]:
        for img in group:
            shutil.copy(f'{base_path}/yes/{img}', f'dataset/{name}/yes/{img}')

    for name, group in [('train', no_train), ('val', no_val), ('test', no_test)]:
        for img in group:
            shutil.copy(f'{base_path}/no/{img}', f'dataset/{name}/no/{img}')

organize_dataset()

# -------------------------------
# 5. LOAD DATA WITH AUGMENTATION
# -------------------------------
IMG_SIZE = 224
BATCH_SIZE = 32

train_aug = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode="nearest"
)

val_aug = ImageDataGenerator(rescale=1./255)

train_data = train_aug.flow_from_directory("dataset/train", target_size=(IMG_SIZE, IMG_SIZE),
                                           batch_size=BATCH_SIZE, class_mode="binary")

val_data = val_aug.flow_from_directory("dataset/val", target_size=(IMG_SIZE, IMG_SIZE),
                                       batch_size=BATCH_SIZE, class_mode="binary")

test_data = val_aug.flow_from_directory("dataset/test", target_size=(IMG_SIZE, IMG_SIZE),
                                        batch_size=BATCH_SIZE, class_mode="binary", shuffle=False)

# -------------------------------
# 6. COMPUTE CLASS WEIGHTS
# -------------------------------
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(train_data.classes),
                                     y=train_data.classes)
class_weights = dict(enumerate(class_weights))

# -------------------------------
# 7. BUILD & FINE-TUNE EfficientNetB0
# -------------------------------
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = True  # fine-tuning entire model

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

# -------------------------------
# 8. TRAINING WITH CALLBACKS
# -------------------------------
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

history = model.fit(train_data,
                    validation_data=val_data,
                    epochs=100,
                    class_weight=class_weights,
                    callbacks=[early_stop, reduce_lr])

# -------------------------------
# 9. PLOT ACCURACY & LOSS
# -------------------------------
plt.figure(figsize=(10,5))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title("Model Accuracy and Loss")
plt.xlabel("Epochs")
plt.ylabel("Value")
plt.legend()
plt.grid(True)
plt.show()

# -------------------------------
# 10. EVALUATION ON TEST SET
# -------------------------------
y_probs = model.predict(test_data)
y_pred = (y_probs > 0.5).astype("int32").reshape(-1)
y_true = test_data.classes

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=["No Tumor", "Tumor"]))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["No Tumor", "Tumor"], yticklabels=["No Tumor", "Tumor"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()


kaggle.json(application/json) - 65 bytes, last modified: 08/05/2025 - 100% done
Saving kaggle.json to kaggle (1).json
Dataset URL: https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection
License(s): copyright-authors
brain-mri-images-for-brain-tumor-detection.zip: Skipping, found more recently modified local copy (use --force to force download)
replace data/brain_tumor_dataset/no/1 no.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Found 161 images belonging to 2 classes.
Found 41 images belonging to 2 classes.
Found 51 images belonging to 2 classes.
Model: "functional_1"
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 112, 112,  │         64 │ block1a_project_… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_conv │ (None, 112, 112,  │      1,536 │ block1a_project_… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_bn   │ (None, 112, 112,  │        384 │ block2a_expand_c… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_act… │ (None, 112, 112,  │          0 │ block2a_expand_b… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_dwconv      │ (None, 56, 56,    │        864 │ block2a_expand_a… │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_bn          │ (None, 56, 56,    │        384 │ block2a_dwconv[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_activation  │ (None, 56, 56,    │          0 │ block2a_bn[0][0]  │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_se_squeeze  │ (None, 96)        │          0 │ block2a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_se_reshape  │ (None, 1, 1, 96)  │          0 │ block2a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_se_reduce   │ (None, 1, 1, 4)   │        388 │ block2a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_se_expand   │ (None, 1, 1, 96)  │        480 │ block2a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_se_excite   │ (None, 56, 56,    │          0 │ block2a_activati… │
│ (Multiply)          │ 96)               │            │ block2a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_project_co… │ (None, 56, 56,    │      2,304 │ block2a_se_excit… │
│ (Conv2D)            │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_project_bn  │ (None, 56, 56,    │         96 │ block2a_project_… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_expand_conv │ (None, 56, 56,    │      3,456 │ block2a_project_… │
│ (Conv2D)            │ 144)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_expand_bn   │ (None, 56, 56,    │        576 │ block2b_expand_c… │
│ (BatchNormalizatio… │ 144)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_expand_act… │ (None, 56, 56,    │          0 │ block2b_expand_b… │
│ (Activation)        │ 144)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_dwconv      │ (None, 56, 56,    │      1,296 │ block2b_expand_a… │
│ (DepthwiseConv2D)   │ 144)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_bn          │ (None, 56, 56,    │        576 │ block2b_dwconv[0… │
│ (BatchNormalizatio… │ 144)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_activation  │ (None, 56, 56,    │          0 │ block2b_bn[0][0]  │
│ (Activation)        │ 144)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_se_squeeze  │ (None, 144)       │          0 │ block2b_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_se_reshape  │ (None, 1, 1, 144) │          0 │ block2b_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_se_reduce   │ (None, 1, 1, 6)   │        870 │ block2b_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_se_expand   │ (None, 1, 1, 144) │      1,008 │ block2b_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_se_excite   │ (None, 56, 56,    │          0 │ block2b_activati… │
│ (Multiply)          │ 144)              │            │ block2b_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_project_co… │ (None, 56, 56,    │      3,456 │ block2b_se_excit… │
│ (Conv2D)            │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_project_bn  │ (None, 56, 56,    │         96 │ block2b_project_… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_drop        │ (None, 56, 56,    │          0 │ block2b_project_… │
│ (FixedDropout)      │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_add (Add)   │ (None, 56, 56,    │          0 │ block2b_drop[0][… │
│                     │ 24)               │            │ block2a_project_… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3a_expand_conv │ (None, 56, 56,    │      3,456 │ block2b_add[0][0] │
│ (Conv2D)            │ 144)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3a_expand_bn   │ (None, 56, 56,    │        576 │ block3a_expand_c… │
│ (BatchNormalizatio… │ 144)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3a_expand_act… │ (None, 56, 56,    │          0 │ block3a_expand_b… │
│ (Activation)        │ 144)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3a_dwconv      │ (None, 28, 28,    │      3,600 │ block3a_expand_a… │
│ (DepthwiseConv2D)   │ 144)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3a_bn          │ (None, 28, 28,    │        576 │ block3a_dwconv[0… │
│ (BatchNormalizatio… │ 144)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3a_activation  │ (None, 28, 28,    │          0 │ block3a_bn[0][0]  │
│ (Activation)        │ 144)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3a_se_squeeze  │ (None, 144)       │          0 │ block3a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3a_se_reshape  │ (None, 1, 1, 144) │          0 │ block3a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3a_se_reduce   │ (None, 1, 1, 6)   │        870 │ block3a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3a_se_expand   │ (None, 1, 1, 144) │      1,008 │ block3a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3a_se_excite   │ (None, 28, 28,    │          0 │ block3a_activati… │
│ (Multiply)          │ 144)              │            │ block3a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3a_project_co… │ (None, 28, 28,    │      5,760 │ block3a_se_excit… │
│ (Conv2D)            │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3a_project_bn  │ (None, 28, 28,    │        160 │ block3a_project_… │
│ (BatchNormalizatio… │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3b_expand_conv │ (None, 28, 28,    │      9,600 │ block3a_project_… │
│ (Conv2D)            │ 240)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3b_expand_bn   │ (None, 28, 28,    │        960 │ block3b_expand_c… │
│ (BatchNormalizatio… │ 240)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3b_expand_act… │ (None, 28, 28,    │          0 │ block3b_expand_b… │
│ (Activation)        │ 240)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3b_dwconv      │ (None, 28, 28,    │      6,000 │ block3b_expand_a… │
│ (DepthwiseConv2D)   │ 240)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3b_bn          │ (None, 28, 28,    │        960 │ block3b_dwconv[0… │
│ (BatchNormalizatio… │ 240)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3b_activation  │ (None, 28, 28,    │          0 │ block3b_bn[0][0]  │
│ (Activation)        │ 240)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3b_se_squeeze  │ (None, 240)       │          0 │ block3b_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3b_se_reshape  │ (None, 1, 1, 240) │          0 │ block3b_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3b_se_reduce   │ (None, 1, 1, 10)  │      2,410 │ block3b_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3b_se_expand   │ (None, 1, 1, 240) │      2,640 │ block3b_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3b_se_excite   │ (None, 28, 28,    │          0 │ block3b_activati… │
│ (Multiply)          │ 240)              │            │ block3b_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3b_project_co… │ (None, 28, 28,    │      9,600 │ block3b_se_excit… │
│ (Conv2D)            │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3b_project_bn  │ (None, 28, 28,    │        160 │ block3b_project_… │
│ (BatchNormalizatio… │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3b_drop        │ (None, 28, 28,    │          0 │ block3b_project_… │
│ (FixedDropout)      │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3b_add (Add)   │ (None, 28, 28,    │          0 │ block3b_drop[0][… │
│                     │ 40)               │            │ block3a_project_… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4a_expand_conv │ (None, 28, 28,    │      9,600 │ block3b_add[0][0] │
│ (Conv2D)            │ 240)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4a_expand_bn   │ (None, 28, 28,    │        960 │ block4a_expand_c… │
│ (BatchNormalizatio… │ 240)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4a_expand_act… │ (None, 28, 28,    │          0 │ block4a_expand_b… │
│ (Activation)        │ 240)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4a_dwconv      │ (None, 14, 14,    │      2,160 │ block4a_expand_a… │
│ (DepthwiseConv2D)   │ 240)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4a_bn          │ (None, 14, 14,    │        960 │ block4a_dwconv[0… │
│ (BatchNormalizatio… │ 240)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4a_activation  │ (None, 14, 14,    │          0 │ block4a_bn[0][0]  │
│ (Activation)        │ 240)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4a_se_squeeze  │ (None, 240)       │          0 │ block4a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4a_se_reshape  │ (None, 1, 1, 240) │          0 │ block4a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4a_se_reduce   │ (None, 1, 1, 10)  │      2,410 │ block4a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4a_se_expand   │ (None, 1, 1, 240) │      2,640 │ block4a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4a_se_excite   │ (None, 14, 14,    │          0 │ block4a_activati… │
│ (Multiply)          │ 240)              │            │ block4a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4a_project_co… │ (None, 14, 14,    │     19,200 │ block4a_se_excit… │
│ (Conv2D)            │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4a_project_bn  │ (None, 14, 14,    │        320 │ block4a_project_… │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4b_expand_conv │ (None, 14, 14,    │     38,400 │ block4a_project_… │
│ (Conv2D)            │ 480)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4b_expand_bn   │ (None, 14, 14,    │      1,920 │ block4b_expand_c… │
│ (BatchNormalizatio… │ 480)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4b_expand_act… │ (None, 14, 14,    │          0 │ block4b_expand_b… │
│ (Activation)        │ 480)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4b_dwconv      │ (None, 14, 14,    │      4,320 │ block4b_expand_a… │
│ (DepthwiseConv2D)   │ 480)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4b_bn          │ (None, 14, 14,    │      1,920 │ block4b_dwconv[0… │
│ (BatchNormalizatio… │ 480)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4b_activation  │ (None, 14, 14,    │          0 │ block4b_bn[0][0]  │
│ (Activation)        │ 480)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4b_se_squeeze  │ (None, 480)       │          0 │ block4b_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4b_se_reshape  │ (None, 1, 1, 480) │          0 │ block4b_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4b_se_reduce   │ (None, 1, 1, 20)  │      9,620 │ block4b_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4b_se_expand   │ (None, 1, 1, 480) │     10,080 │ block4b_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4b_se_excite   │ (None, 14, 14,    │          0 │ block4b_activati… │
│ (Multiply)          │ 480)              │            │ block4b_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4b_project_co… │ (None, 14, 14,    │     38,400 │ block4b_se_excit… │
│ (Conv2D)            │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4b_project_bn  │ (None, 14, 14,    │        320 │ block4b_project_… │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4b_drop        │ (None, 14, 14,    │          0 │ block4b_project_… │
│ (FixedDropout)      │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4b_add (Add)   │ (None, 14, 14,    │          0 │ block4b_drop[0][… │
│                     │ 80)               │            │ block4a_project_… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4c_expand_conv │ (None, 14, 14,    │     38,400 │ block4b_add[0][0] │
│ (Conv2D)            │ 480)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4c_expand_bn   │ (None, 14, 14,    │      1,920 │ block4c_expand_c… │
│ (BatchNormalizatio… │ 480)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4c_expand_act… │ (None, 14, 14,    │          0 │ block4c_expand_b… │
│ (Activation)        │ 480)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4c_dwconv      │ (None, 14, 14,    │      4,320 │ block4c_expand_a… │
│ (DepthwiseConv2D)   │ 480)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4c_bn          │ (None, 14, 14,    │      1,920 │ block4c_dwconv[0… │
│ (BatchNormalizatio… │ 480)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4c_activation  │ (None, 14, 14,    │          0 │ block4c_bn[0][0]  │
│ (Activation)        │ 480)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4c_se_squeeze  │ (None, 480)       │          0 │ block4c_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4c_se_reshape  │ (None, 1, 1, 480) │          0 │ block4c_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4c_se_reduce   │ (None, 1, 1, 20)  │      9,620 │ block4c_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4c_se_expand   │ (None, 1, 1, 480) │     10,080 │ block4c_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4c_se_excite   │ (None, 14, 14,    │          0 │ block4c_activati… │
│ (Multiply)          │ 480)              │            │ block4c_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4c_project_co… │ (None, 14, 14,    │     38,400 │ block4c_se_excit… │
│ (Conv2D)            │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4c_project_bn  │ (None, 14, 14,    │        320 │ block4c_project_… │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4c_drop        │ (None, 14, 14,    │          0 │ block4c_project_… │
│ (FixedDropout)      │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4c_add (Add)   │ (None, 14, 14,    │          0 │ block4c_drop[0][… │
│                     │ 80)               │            │ block4b_add[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5a_expand_conv │ (None, 14, 14,    │     38,400 │ block4c_add[0][0] │
│ (Conv2D)            │ 480)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5a_expand_bn   │ (None, 14, 14,    │      1,920 │ block5a_expand_c… │
│ (BatchNormalizatio… │ 480)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5a_expand_act… │ (None, 14, 14,    │          0 │ block5a_expand_b… │
│ (Activation)        │ 480)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5a_dwconv      │ (None, 14, 14,    │     12,000 │ block5a_expand_a… │
│ (DepthwiseConv2D)   │ 480)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5a_bn          │ (None, 14, 14,    │      1,920 │ block5a_dwconv[0… │
│ (BatchNormalizatio… │ 480)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5a_activation  │ (None, 14, 14,    │          0 │ block5a_bn[0][0]  │
│ (Activation)        │ 480)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5a_se_squeeze  │ (None, 480)       │          0 │ block5a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5a_se_reshape  │ (None, 1, 1, 480) │          0 │ block5a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5a_se_reduce   │ (None, 1, 1, 20)  │      9,620 │ block5a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5a_se_expand   │ (None, 1, 1, 480) │     10,080 │ block5a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5a_se_excite   │ (None, 14, 14,    │          0 │ block5a_activati… │
│ (Multiply)          │ 480)              │            │ block5a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5a_project_co… │ (None, 14, 14,    │     53,760 │ block5a_se_excit… │
│ (Conv2D)            │ 112)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5a_project_bn  │ (None, 14, 14,    │        448 │ block5a_project_… │
│ (BatchNormalizatio… │ 112)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5b_expand_conv │ (None, 14, 14,    │     75,264 │ block5a_project_… │
│ (Conv2D)            │ 672)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5b_expand_bn   │ (None, 14, 14,    │      2,688 │ block5b_expand_c… │
│ (BatchNormalizatio… │ 672)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5b_expand_act… │ (None, 14, 14,    │          0 │ block5b_expand_b… │
│ (Activation)        │ 672)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5b_dwconv      │ (None, 14, 14,    │     16,800 │ block5b_expand_a… │
│ (DepthwiseConv2D)   │ 672)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5b_bn          │ (None, 14, 14,    │      2,688 │ block5b_dwconv[0… │
│ (BatchNormalizatio… │ 672)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5b_activation  │ (None, 14, 14,    │          0 │ block5b_bn[0][0]  │
│ (Activation)        │ 672)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5b_se_squeeze  │ (None, 672)       │          0 │ block5b_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5b_se_reshape  │ (None, 1, 1, 672) │          0 │ block5b_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5b_se_reduce   │ (None, 1, 1, 28)  │     18,844 │ block5b_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5b_se_expand   │ (None, 1, 1, 672) │     19,488 │ block5b_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5b_se_excite   │ (None, 14, 14,    │          0 │ block5b_activati… │
│ (Multiply)          │ 672)              │            │ block5b_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5b_project_co… │ (None, 14, 14,    │     75,264 │ block5b_se_excit… │
│ (Conv2D)            │ 112)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5b_project_bn  │ (None, 14, 14,    │        448 │ block5b_project_… │
│ (BatchNormalizatio… │ 112)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5b_drop        │ (None, 14, 14,    │          0 │ block5b_project_… │
│ (FixedDropout)      │ 112)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5b_add (Add)   │ (None, 14, 14,    │          0 │ block5b_drop[0][… │
│                     │ 112)              │            │ block5a_project_… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5c_expand_conv │ (None, 14, 14,    │     75,264 │ block5b_add[0][0] │
│ (Conv2D)            │ 672)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5c_expand_bn   │ (None, 14, 14,    │      2,688 │ block5c_expand_c… │
│ (BatchNormalizatio… │ 672)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5c_expand_act… │ (None, 14, 14,    │          0 │ block5c_expand_b… │
│ (Activation)        │ 672)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5c_dwconv      │ (None, 14, 14,    │     16,800 │ block5c_expand_a… │
│ (DepthwiseConv2D)   │ 672)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5c_bn          │ (None, 14, 14,    │      2,688 │ block5c_dwconv[0… │
│ (BatchNormalizatio… │ 672)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5c_activation  │ (None, 14, 14,    │          0 │ block5c_bn[0][0]  │
│ (Activation)        │ 672)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5c_se_squeeze  │ (None, 672)       │          0 │ block5c_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5c_se_reshape  │ (None, 1, 1, 672) │          0 │ block5c_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5c_se_reduce   │ (None, 1, 1, 28)  │     18,844 │ block5c_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5c_se_expand   │ (None, 1, 1, 672) │     19,488 │ block5c_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5c_se_excite   │ (None, 14, 14,    │          0 │ block5c_activati… │
│ (Multiply)          │ 672)              │            │ block5c_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5c_project_co… │ (None, 14, 14,    │     75,264 │ block5c_se_excit… │
│ (Conv2D)            │ 112)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5c_project_bn  │ (None, 14, 14,    │        448 │ block5c_project_… │
│ (BatchNormalizatio… │ 112)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5c_drop        │ (None, 14, 14,    │          0 │ block5c_project_… │
│ (FixedDropout)      │ 112)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5c_add (Add)   │ (None, 14, 14,    │          0 │ block5c_drop[0][… │
│                     │ 112)              │            │ block5b_add[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6a_expand_conv │ (None, 14, 14,    │     75,264 │ block5c_add[0][0] │
│ (Conv2D)            │ 672)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6a_expand_bn   │ (None, 14, 14,    │      2,688 │ block6a_expand_c… │
│ (BatchNormalizatio… │ 672)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6a_expand_act… │ (None, 14, 14,    │          0 │ block6a_expand_b… │
│ (Activation)        │ 672)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6a_dwconv      │ (None, 7, 7, 672) │     16,800 │ block6a_expand_a… │
│ (DepthwiseConv2D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6a_bn          │ (None, 7, 7, 672) │      2,688 │ block6a_dwconv[0… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6a_activation  │ (None, 7, 7, 672) │          0 │ block6a_bn[0][0]  │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6a_se_squeeze  │ (None, 672)       │          0 │ block6a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6a_se_reshape  │ (None, 1, 1, 672) │          0 │ block6a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6a_se_reduce   │ (None, 1, 1, 28)  │     18,844 │ block6a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6a_se_expand   │ (None, 1, 1, 672) │     19,488 │ block6a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6a_se_excite   │ (None, 7, 7, 672) │          0 │ block6a_activati… │
│ (Multiply)          │                   │            │ block6a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6a_project_co… │ (None, 7, 7, 192) │    129,024 │ block6a_se_excit… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6a_project_bn  │ (None, 7, 7, 192) │        768 │ block6a_project_… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6b_expand_conv │ (None, 7, 7,      │    221,184 │ block6a_project_… │
│ (Conv2D)            │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6b_expand_bn   │ (None, 7, 7,      │      4,608 │ block6b_expand_c… │
│ (BatchNormalizatio… │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6b_expand_act… │ (None, 7, 7,      │          0 │ block6b_expand_b… │
│ (Activation)        │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6b_dwconv      │ (None, 7, 7,      │     28,800 │ block6b_expand_a… │
│ (DepthwiseConv2D)   │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6b_bn          │ (None, 7, 7,      │      4,608 │ block6b_dwconv[0… │
│ (BatchNormalizatio… │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6b_activation  │ (None, 7, 7,      │          0 │ block6b_bn[0][0]  │
│ (Activation)        │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6b_se_squeeze  │ (None, 1152)      │          0 │ block6b_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6b_se_reshape  │ (None, 1, 1,      │          0 │ block6b_se_squee… │
│ (Reshape)           │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6b_se_reduce   │ (None, 1, 1, 48)  │     55,344 │ block6b_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6b_se_expand   │ (None, 1, 1,      │     56,448 │ block6b_se_reduc… │
│ (Conv2D)            │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6b_se_excite   │ (None, 7, 7,      │          0 │ block6b_activati… │
│ (Multiply)          │ 1152)             │            │ block6b_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6b_project_co… │ (None, 7, 7, 192) │    221,184 │ block6b_se_excit… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6b_project_bn  │ (None, 7, 7, 192) │        768 │ block6b_project_… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6b_drop        │ (None, 7, 7, 192) │          0 │ block6b_project_… │
│ (FixedDropout)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6b_add (Add)   │ (None, 7, 7, 192) │          0 │ block6b_drop[0][… │
│                     │                   │            │ block6a_project_… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6c_expand_conv │ (None, 7, 7,      │    221,184 │ block6b_add[0][0] │
│ (Conv2D)            │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6c_expand_bn   │ (None, 7, 7,      │      4,608 │ block6c_expand_c… │
│ (BatchNormalizatio… │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6c_expand_act… │ (None, 7, 7,      │          0 │ block6c_expand_b… │
│ (Activation)        │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6c_dwconv      │ (None, 7, 7,      │     28,800 │ block6c_expand_a… │
│ (DepthwiseConv2D)   │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6c_bn          │ (None, 7, 7,      │      4,608 │ block6c_dwconv[0… │
│ (BatchNormalizatio… │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6c_activation  │ (None, 7, 7,      │          0 │ block6c_bn[0][0]  │
│ (Activation)        │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6c_se_squeeze  │ (None, 1152)      │          0 │ block6c_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6c_se_reshape  │ (None, 1, 1,      │          0 │ block6c_se_squee… │
│ (Reshape)           │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6c_se_reduce   │ (None, 1, 1, 48)  │     55,344 │ block6c_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6c_se_expand   │ (None, 1, 1,      │     56,448 │ block6c_se_reduc… │
│ (Conv2D)            │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6c_se_excite   │ (None, 7, 7,      │          0 │ block6c_activati… │
│ (Multiply)          │ 1152)             │            │ block6c_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6c_project_co… │ (None, 7, 7, 192) │    221,184 │ block6c_se_excit… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6c_project_bn  │ (None, 7, 7, 192) │        768 │ block6c_project_… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6c_drop        │ (None, 7, 7, 192) │          0 │ block6c_project_… │
│ (FixedDropout)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6c_add (Add)   │ (None, 7, 7, 192) │          0 │ block6c_drop[0][… │
│                     │                   │            │ block6b_add[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6d_expand_conv │ (None, 7, 7,      │    221,184 │ block6c_add[0][0] │
│ (Conv2D)            │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6d_expand_bn   │ (None, 7, 7,      │      4,608 │ block6d_expand_c… │
│ (BatchNormalizatio… │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6d_expand_act… │ (None, 7, 7,      │          0 │ block6d_expand_b… │
│ (Activation)        │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6d_dwconv      │ (None, 7, 7,      │     28,800 │ block6d_expand_a… │
│ (DepthwiseConv2D)   │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6d_bn          │ (None, 7, 7,      │      4,608 │ block6d_dwconv[0… │
│ (BatchNormalizatio… │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6d_activation  │ (None, 7, 7,      │          0 │ block6d_bn[0][0]  │
│ (Activation)        │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6d_se_squeeze  │ (None, 1152)      │          0 │ block6d_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6d_se_reshape  │ (None, 1, 1,      │          0 │ block6d_se_squee… │
│ (Reshape)           │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6d_se_reduce   │ (None, 1, 1, 48)  │     55,344 │ block6d_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6d_se_expand   │ (None, 1, 1,      │     56,448 │ block6d_se_reduc… │
│ (Conv2D)            │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6d_se_excite   │ (None, 7, 7,      │          0 │ block6d_activati… │
│ (Multiply)          │ 1152)             │            │ block6d_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6d_project_co… │ (None, 7, 7, 192) │    221,184 │ block6d_se_excit… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6d_project_bn  │ (None, 7, 7, 192) │        768 │ block6d_project_… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6d_drop        │ (None, 7, 7, 192) │          0 │ block6d_project_… │
│ (FixedDropout)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block6d_add (Add)   │ (None, 7, 7, 192) │          0 │ block6d_drop[0][… │
│                     │                   │            │ block6c_add[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block7a_expand_conv │ (None, 7, 7,      │    221,184 │ block6d_add[0][0] │
│ (Conv2D)            │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block7a_expand_bn   │ (None, 7, 7,      │      4,608 │ block7a_expand_c… │
│ (BatchNormalizatio… │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block7a_expand_act… │ (None, 7, 7,      │          0 │ block7a_expand_b… │
│ (Activation)        │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block7a_dwconv      │ (None, 7, 7,      │     10,368 │ block7a_expand_a… │
│ (DepthwiseConv2D)   │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block7a_bn          │ (None, 7, 7,      │      4,608 │ block7a_dwconv[0… │
│ (BatchNormalizatio… │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block7a_activation  │ (None, 7, 7,      │          0 │ block7a_bn[0][0]  │
│ (Activation)        │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block7a_se_squeeze  │ (None, 1152)      │          0 │ block7a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block7a_se_reshape  │ (None, 1, 1,      │          0 │ block7a_se_squee… │
│ (Reshape)           │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block7a_se_reduce   │ (None, 1, 1, 48)  │     55,344 │ block7a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block7a_se_expand   │ (None, 1, 1,      │     56,448 │ block7a_se_reduc… │
│ (Conv2D)            │ 1152)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block7a_se_excite   │ (None, 7, 7,      │          0 │ block7a_activati… │
│ (Multiply)          │ 1152)             │            │ block7a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block7a_project_co… │ (None, 7, 7, 320) │    368,640 │ block7a_se_excit… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block7a_project_bn  │ (None, 7, 7, 320) │      1,280 │ block7a_project_… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ top_conv (Conv2D)   │ (None, 7, 7,      │    409,600 │ block7a_project_… │
│                     │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ top_bn              │ (None, 7, 7,      │      5,120 │ top_conv[0][0]    │
│ (BatchNormalizatio… │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ top_activation      │ (None, 7, 7,      │          0 │ top_bn[0][0]      │
│ (Activation)        │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1280)      │          0 │ top_activation[0… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 1280)      │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 128)       │    163,968 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 128)       │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │        129 │ dropout_3[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘
 Total params: 4,213,661 (16.07 MB)
 Trainable params: 4,171,645 (15.91 MB)
 Non-trainable params: 42,016 (164.12 KB)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 106s 7s/step - accuracy: 0.4404 - loss: 0.7375 - val_accuracy: 0.6585 - val_loss: 0.6763 - learning_rate: 1.0000e-04
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 15s 387ms/step - accuracy: 0.6547 - loss: 0.6423 - val_accuracy: 0.6829 - val_loss: 0.6737 - learning_rate: 1.0000e-04
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 380ms/step - accuracy: 0.6125 - loss: 0.6235 - val_accuracy: 0.5854 - val_loss: 0.6679 - learning_rate: 1.0000e-04
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 386ms/step - accuracy: 0.7110 - loss: 0.6022 - val_accuracy: 0.6341 - val_loss: 0.6567 - learning_rate: 1.0000e-04
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 386ms/step - accuracy: 0.7078 - loss: 0.5447 - val_accuracy: 0.6341 - val_loss: 0.6436 - learning_rate: 1.0000e-04
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 543ms/step - accuracy: 0.8067 - loss: 0.4983 - val_accuracy: 0.6829 - val_loss: 0.6264 - learning_rate: 1.0000e-04
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 390ms/step - accuracy: 0.8269 - loss: 0.4476 - val_accuracy: 0.7317 - val_loss: 0.6077 - learning_rate: 1.0000e-04
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 475ms/step - accuracy: 0.8989 - loss: 0.4090 - val_accuracy: 0.7317 - val_loss: 0.5915 - learning_rate: 1.0000e-04
Epoch 9/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 384ms/step - accuracy: 0.8912 - loss: 0.3818 - val_accuracy: 0.7317 - val_loss: 0.5852 - learning_rate: 1.0000e-04
Epoch 10/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 471ms/step - accuracy: 0.9238 - loss: 0.3387 - val_accuracy: 0.7561 - val_loss: 0.5729 - learning_rate: 1.0000e-04
Epoch 11/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 482ms/step - accuracy: 0.9004 - loss: 0.3564 - val_accuracy: 0.7805 - val_loss: 0.5541 - learning_rate: 1.0000e-04
Epoch 12/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 379ms/step - accuracy: 0.8717 - loss: 0.2924 - val_accuracy: 0.7805 - val_loss: 0.5283 - learning_rate: 1.0000e-04
Epoch 13/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 388ms/step - accuracy: 0.9192 - loss: 0.2760 - val_accuracy: 0.7561 - val_loss: 0.4953 - learning_rate: 1.0000e-04
Epoch 14/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 559ms/step - accuracy: 0.9056 - loss: 0.2907 - val_accuracy: 0.7561 - val_loss: 0.4637 - learning_rate: 1.0000e-04
Epoch 15/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 384ms/step - accuracy: 0.9408 - loss: 0.2292 - val_accuracy: 0.7805 - val_loss: 0.4358 - learning_rate: 1.0000e-04
Epoch 16/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 465ms/step - accuracy: 0.8049 - loss: 0.4096 - val_accuracy: 0.8049 - val_loss: 0.4336 - learning_rate: 1.0000e-04
Epoch 17/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 379ms/step - accuracy: 0.9816 - loss: 0.1645 - val_accuracy: 0.8049 - val_loss: 0.4345 - learning_rate: 1.0000e-04
Epoch 18/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 422ms/step - accuracy: 0.9884 - loss: 0.1707 - val_accuracy: 0.8293 - val_loss: 0.4249 - learning_rate: 1.0000e-04
Epoch 19/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 596ms/step - accuracy: 0.9630 - loss: 0.2035 - val_accuracy: 0.8293 - val_loss: 0.3973 - learning_rate: 1.0000e-04
Epoch 20/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 374ms/step - accuracy: 0.9723 - loss: 0.1359 - val_accuracy: 0.8293 - val_loss: 0.3635 - learning_rate: 1.0000e-04
Epoch 21/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 386ms/step - accuracy: 0.9733 - loss: 0.1235 - val_accuracy: 0.8293 - val_loss: 0.3326 - learning_rate: 1.0000e-04
Epoch 22/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 463ms/step - accuracy: 0.9801 - loss: 0.1379 - val_accuracy: 0.8537 - val_loss: 0.3026 - learning_rate: 1.0000e-04
Epoch 23/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 381ms/step - accuracy: 0.9517 - loss: 0.1198 - val_accuracy: 0.8293 - val_loss: 0.3162 - learning_rate: 1.0000e-04
Epoch 24/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 371ms/step - accuracy: 0.9714 - loss: 0.1060 - val_accuracy: 0.8293 - val_loss: 0.4010 - learning_rate: 1.0000e-04
Epoch 25/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 365ms/step - accuracy: 0.9855 - loss: 0.1109 - val_accuracy: 0.8293 - val_loss: 0.4835 - learning_rate: 1.0000e-04
Epoch 26/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 501ms/step - accuracy: 0.9520 - loss: 0.1224 - val_accuracy: 0.8293 - val_loss: 0.5084 - learning_rate: 5.0000e-05
Epoch 27/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 361ms/step - accuracy: 0.9735 - loss: 0.1335 - val_accuracy: 0.7805 - val_loss: 0.5678 - learning_rate: 5.0000e-05

2/2 ━━━━━━━━━━━━━━━━━━━━ 11s 6s/step
Classification Report:
              precision    recall  f1-score   support

    No Tumor       0.80      1.00      0.89        20
       Tumor       1.00      0.84      0.91        31

    accuracy                           0.90        51
   macro avg       0.90      0.92      0.90        51
weighted avg       0.92      0.90      0.90        51
